In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/fashion')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Define generator and discriminator

In [0]:
tf.reset_default_graph()

def sample_Z(batch_size, latent_dim):
    return np.random.normal(0,1,size=[batch_size, latent_dim])
#     return np.random.uniform(-1., 1., size=[batch_size, latent_dim])

def sample_data(batch_size, iter):
#     if iter*batch_size+batch_size == 60000

    return train_images[iter*batch_size:iter*batch_size+batch_size,:,:].reshape(batch_size, 784)
  

def generator(Z, hsize=[128, 128], reuse=False):
    with tf.variable_scope("Generator",reuse=reuse):
        h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.relu)
        h2 = tf.layers.dense(h1,hsize[0],activation=tf.nn.relu) 
        h2_drop = tf.nn.dropout(h2,rate=0.3)
        h3 = tf.layers.dense(h2_drop, hsize[1],activation=tf.nn.relu) 
        h4 = tf.layers.dense(h3,hsize[1],activation=tf.nn.relu) 
        h4_drop = tf.nn.dropout(h4,rate=0.3)
        out = tf.layers.dense(h4_drop,784)
    return out


def discriminator(X ,hsize=[128, 128] ,reuse=False):
    with tf.variable_scope("Discriminator",reuse=reuse):
        h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.relu)
        h3 = tf.layers.dense(h2,1)
        out = tf.nn.sigmoid( h3 ) # transform to probability
    return out, h3


# Build Graph for GAN

In [3]:
X = tf.placeholder(tf.float32,[None,784])
Z = tf.placeholder(tf.float32,[None,100])

G_sample = generator(Z)
prob_real, prob_logit_real = discriminator(X)
prob_fake, prob_logit_fake = discriminator(G_sample,reuse=True)

disc_loss = -tf.reduce_mean(tf.log(prob_real) + tf.log(1-prob_fake))
gen_loss = -tf.reduce_mean(tf.log(prob_fake))

print(prob_real.shape, G_sample.shape)

gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Discriminator")

gen_opt = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(gen_loss,var_list = gen_vars) # G Train step
disc_opt = tf.train.AdamOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) 

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
(TensorShape([Dimension(None), Dimension(1)]), TensorShape([Dimension(None), Dimension(784)]))


# Training 

In [0]:
batch_size = 32
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(100001):
    X_batch = data.train.next_batch(batch_size)[0]
    Z_batch = sample_Z(batch_size, 100)
    _, disloss, generated_sample = sess.run([disc_opt, disc_loss, G_sample], feed_dict={X: X_batch, Z: Z_batch})
    _, genloss = sess.run([gen_opt, gen_loss], feed_dict={Z: Z_batch})
    if i % 1000 ==0:
      print(disloss, genloss)
      generated_sample = generated_sample.reshape(32, 28, 28)
      plt.figure(figsize=(10,10))
      for j in range(25):
          plt.subplot(5,5,j+1)
          plt.xticks([])
          plt.yticks([])
          plt.grid(False)
          plt.imshow(generated_sample[j,:,:])



(1.2114043, 0.8294209)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
(nan, nan)
